# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
mens = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
mens['League'] = 'M'
mens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,...,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000
mean,2014.051044,70.072462,1288.243422,75.859651,1283.044987,63.857732,0.068658,26.392099,55.746305,7.339085,...,20.138276,12.072488,17.736907,10.480668,21.632934,11.405867,13.737130,5.901031,3.144239,19.324709
std,6.464256,35.845605,105.298971,11.007412,104.764160,10.851210,0.305052,4.683480,7.461328,3.116574,...,6.064958,5.345290,7.085348,4.221941,4.519345,3.724047,4.536147,2.778302,2.628125,4.551727
min,2003.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,0.000000,10.000000,26.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,2009.000000,38.000000,1199.000000,68.000000,1192.000000,57.000000,0.000000,23.000000,51.000000,5.000000,...,16.000000,8.000000,13.000000,7.000000,19.000000,9.000000,11.000000,4.000000,1.000000,16.000000
50%,2014.000000,73.000000,1287.000000,75.000000,1282.000000,64.000000,0.000000,26.000000,55.000000,7.000000,...,20.000000,12.000000,17.000000,10.000000,21.000000,11.000000,13.000000,6.000000,3.000000,19.000000
75%,2020.000000,101.000000,1381.000000,83.000000,1374.000000,71.000000,0.000000,29.000000,60.000000,9.000000,...,24.000000,15.000000,22.000000,13.000000,25.000000,14.000000,17.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,1480.000000,149.000000,1480.000000,144.000000,6.000000,57.000000,103.000000,26.000000,...,59.000000,48.000000,65.000000,36.000000,49.000000,31.000000,41.000000,22.000000,33.000000,45.000000


In [3]:
womens = pd.read_csv('data/WRegularSeasonDetailedResults.csv')
womens['League'] = 'W'
womens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,...,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000
mean,2017.310476,68.860759,3285.050867,71.711963,3286.594658,57.234370,0.051583,25.847537,58.980010,6.268876,...,17.913974,10.511119,15.515175,11.395447,22.441116,10.933688,16.745024,6.923831,3.434950,18.204184
std,4.531798,36.258086,104.022507,11.547894,105.457243,10.964583,0.258755,4.982451,7.975729,3.125925,...,6.469817,4.938106,6.632564,4.639725,4.939763,3.805204,5.597689,3.279905,3.666537,4.557235
min,2010.000000,0.000000,3101.000000,30.000000,3101.000000,11.000000,0.000000,9.000000,30.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2013.000000,35.000000,3196.000000,64.000000,3195.000000,50.000000,0.000000,22.000000,53.000000,4.000000,...,13.000000,7.000000,11.000000,8.000000,19.000000,8.000000,13.000000,5.000000,1.000000,15.000000
50%,2017.000000,72.000000,3283.000000,71.000000,3287.000000,57.000000,0.000000,25.000000,59.000000,6.000000,...,17.000000,10.000000,15.000000,11.000000,22.000000,11.000000,16.000000,7.000000,3.000000,18.000000
75%,2021.000000,101.000000,3376.000000,79.000000,3377.000000,64.000000,0.000000,29.000000,64.000000,8.000000,...,22.000000,14.000000,20.000000,14.000000,26.000000,13.000000,20.000000,9.000000,4.000000,21.000000
max,2025.000000,132.000000,3480.000000,140.000000,3480.000000,130.000000,5.000000,58.000000,113.000000,30.000000,...,80.000000,37.000000,52.000000,38.000000,53.000000,34.000000,49.000000,26.000000,42.000000,47.000000


The IDs are definitely distinct so we can combine into a single dataframe

In [4]:
all_teams = pd.concat([mens, womens])

Get the distinct team/Years

In [5]:
teams = pd.concat([all_teams[['WTeamID', 'Season', 'League']].rename(columns={'WTeamID': 'TeamID'}),
                   all_teams[['LTeamID', 'Season', 'League']].rename(columns={'LTeamID': 'TeamID'})]).drop_duplicates().reset_index()

Define the training data. The x's will be the indexes of two team IDs, the y's will be 1 if the first team won, 0 otherwise.

In [6]:
teamMapping = {(x.TeamID, x.Season): x.Index for x in teams.itertuples()}

In [7]:
winners = all_teams.apply(lambda x: teamMapping[(x.WTeamID, x.Season)], axis=1)
losers = all_teams.apply(lambda x: teamMapping[(x.LTeamID, x.Season)], axis=1)

In [8]:
x_tensor = torch.from_numpy(np.concatenate([np.stack([winners, losers], axis=1), np.stack([losers, winners], axis=1)]))
y_tensor = torch.from_numpy(np.concatenate([np.ones(len(all_teams)), np.zeros(len(all_teams))])).reshape([-1,1])
dataset = TensorDataset(x_tensor, y_tensor)

Generate the train/validation split

In [9]:
generator = torch.Generator().manual_seed(20250217)
train_data, validation_data = torch.utils.data.random_split(dataset, [0.9, 0.1], generator=generator)
train_loader = DataLoader(train_data, batch_size=64)
validation_loader = DataLoader(validation_data, batch_size=64)

## The Model
Define the model. Combine the embeddings for the two teams, go to a hidden layer, and then output to a prediction if the first team won

In [10]:
class Model(nn.Module):
    def __init__(self, embedding_size=64, model_size=16, dropout=0.1):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(len(teams), embedding_size)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(2*embedding_size, model_size)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(model_size, 1)
        self.double()

    def forward(self, x):
        team = self.embedding(x[:,0])
        opponent = self.embedding(x[:,1])
        matchup = self.dropout1(torch.cat([team, opponent], axis=1))
        hidden = self.dropout2(F.relu(self.fc1(matchup)))
        result = self.fc2(hidden)
        return result
        

In [11]:

model = Model(embedding_size=128, model_size=8).to(device)

## Training the model

Define the training function

In [12]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

def train(data, model, loss_fn, optimizer, log=False):
    size = len(data.dataset)
    model.train()
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if log and batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss {loss:>7f} [{current:>6d}/{size:>6d}]")

Define the testing function

In [13]:
def test(data, model, loss_fn, label="Test"):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in data:
            x = x.to(device)
            y = y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += ((pred >= 0.5) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"{label} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

Train the model

In [14]:
for i in range(5):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer, log=False)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train Error: Accuracy: 70.3%, Avg loss: 0.198101
Validation Error: Accuracy: 67.5%, Avg loss: 0.207310
Epoch 1
Train Error: Accuracy: 76.2%, Avg loss: 0.166931
Validation Error: Accuracy: 73.4%, Avg loss: 0.179716
Epoch 2
Train Error: Accuracy: 77.2%, Avg loss: 0.160403
Validation Error: Accuracy: 74.4%, Avg loss: 0.173460
Epoch 3
Train Error: Accuracy: 77.5%, Avg loss: 0.158391
Validation Error: Accuracy: 74.7%, Avg loss: 0.171679
Epoch 4
Train Error: Accuracy: 77.5%, Avg loss: 0.158235
Validation Error: Accuracy: 74.6%, Avg loss: 0.171791


With this model we can predicct the output of about three quarters of regular season games.

## Load the tourney data to test with

In [15]:
mens_tourney = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
womens_tourney = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
tourney = pd.concat([mens_tourney, womens_tourney])
tourney_winners = tourney.apply(lambda x: teamMapping[(x.WTeamID, x.Season)], axis=1)
tourney_losers = tourney.apply(lambda x: teamMapping[(x.LTeamID, x.Season)], axis=1)
x_tourney_tensor = torch.from_numpy(np.concatenate([np.stack([tourney_winners, tourney_losers], axis=1), 
                                                    np.stack([tourney_losers, tourney_winners], axis=1)]))
y_tourney_tensor = torch.from_numpy(np.concatenate([np.ones(len(tourney)), np.zeros(len(tourney))])).reshape([-1,1])
tourney_dataset = TensorDataset(x_tourney_tensor, y_tourney_tensor)
tourney_loader = DataLoader(tourney_dataset, batch_size=64)

In [16]:
test(tourney_loader, model, loss_fn, label="Tourney")

Tourney Error: Accuracy: 69.7%, Avg loss: 0.188228


When it comes to tournament results we get about 7 out of 10 results. The lower result is likely due to teams having increased pairity.

## Generating the submission file
### Phase 1

Write the results

In [17]:
with open('submission.csv', 'w') as f:
    f.write("ID,Pred\n")
    for season in range(2021, 2025):
        for league in ('M', 'W'):
            teams_to_test = sorted(teams[(teams.Season==season) & (teams.League==league)].TeamID.values)
            matchups = [(t1, t2) for t1 in teams_to_test for t2 in teams_to_test if t1 < t2]
            matchups_tensor = torch.Tensor([(teamMapping[(t1, season)], teamMapping[(t2, season)])
                                     for (t1, t2) in matchups]).int().to(device)
            predictions = model(matchups_tensor)
            for (t1, t2), pred in zip(matchups, predictions):
                f.write(f"{season}_{t1.item()}_{t2.item()},{pred.item()}\n")

Two teams canceled their 2021 season due to covid but are still in the sample submission. Add in their results

In [18]:
with open('submission.csv', 'a') as f:
    for missing_team in [3169, 3197]:
        for opponent in teams[(teams.Season==2021) & (teams.League=='W')].TeamID.values:
            if opponent > missing_team:
                f.write(f"2021_{missing_team}_{opponent},0\n")
            else:
                f.write(f"2021_{opponent}_{missing_team},1\n")
    f.write(f"2021_3169_3197,0.5\n")